<a href="https://colab.research.google.com/github/Maram-Elsayed/Flower-Recognizer-in-Python/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from skimage import feature
import numpy as np
import PIL
import os, os.path
import cv2
from skimage.feature import local_binary_pattern



In [0]:
import random
from scipy.stats import itemfreq
imageDir_train = ["drive/Colab/daisy_0/","drive/Colab/dandelion_1/","drive/Colab/rose_2/","drive/Colab/sunflower_3/","drive/Colab/tulip_4/"]

y=0
data = []
labels = []
radius = 11
no_points =24
image_path_list_train=[]
yt=[]
train_y=np.array([])
valid_image_extensions = [".jpg", ".jpeg", ".png"]
valid_image_extensions = [item.lower() for item in valid_image_extensions]
for f in range(len(imageDir_train)): 

 for file in os.listdir(imageDir_train[f]):    
    extension = os.path.splitext(file)[1]
    if extension.lower() not in valid_image_extensions:
        continue
    image_path_list_train.append(os.path.join(imageDir_train[f], file))
    
    
random.shuffle(image_path_list_train)
for imagePath in image_path_list_train:
   
   i= cv2.imread(imagePath)
   gray = cv2.cvtColor(i,cv2.COLOR_BGR2GRAY)
   res = cv2.resize(gray, dsize=(320, 230), interpolation=cv2.INTER_CUBIC)
   lbp = local_binary_pattern(res, no_points, radius, method='uniform')
   x = itemfreq(lbp.ravel())
   hist = x[:, 1]/sum(x[:, 1])
   data.append(hist)
   labels.append(imagePath.split("/")[-2])
   y+=1
   



In [0]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.10)

In [0]:
  
from sklearn.svm import SVC  
model = SVC(C=120, gamma=1,random_state=42)
model.fit(X_train, y_train)  

SVC(C=120, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [0]:
y_pred = model.predict(X_test) 

In [0]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 